In [ ]:
# -*- coding: utf-8 -*-
# install libraries
%pip install --upgrade tiktoken
%pip install --upgrade openai
%pip install --upgrade python-dotenv
%pip install --upgrade pydub
%pip install --upgrade soundfile
%pip install --upgrade openai-whisper

In [29]:
# import libraries
import os
import re
import time
import json
import base64
import pprint
import tiktoken
import tempfile
import subprocess
import concurrent.futures
from dotenv import load_dotenv
from openai import AzureOpenAI
import openai
import pandas as pd

# Funções que serão utilizadas durante os exemplos

In [36]:
# Nome das regiões Azure OpenAI
EASTUS = "eastus"
CANADAEAST = "canadaeast"
NORTHCENTRALUS = "northcentralus"

# Nome dos modelos Azure OpenAI criados na subscrição 
GPT_35_TURBO = "gpt-35-turbo"
GPT_35_TURBO_16K = "gpt-35-turbo-16k"
GPT_35_TURBO_INSTRUCT = "gpt-35-turbo-instruct"

GPT_4 = "gpt-4"
GPT_4_32K = "gpt-4-32k"
GPT_4_TURBO = "gpt-4-turbo"
GPT_4o = "gpt-4o"
GPT_4o_mini = "gpt-4o-mini"

DALL_E_3 = "dall-e-3"
WHISPER = "whisper"
TEXT_EMBEDDING_ADA_002 = "text-embedding-ada-002"
TEXT_EMBEDDING_ADA_3_SMALL = "text-embedding-3-small"
TEXT_EMBEDDING_ADA_3_LARGE = "text-embedding-3-large"

# Whisper limit 25MB files
audio_chunk_size_kb = 1024 * 22


load_dotenv() # carregar variáveis de ambiente

# gpt-4o em northcentralus 2024-05-13
# gpt-4o em eastus 2024-08-06

deployments_in_regions = {"CANADAEAST": [GPT_35_TURBO, GPT_4, GPT_4_32K, GPT_4_TURBO, TEXT_EMBEDDING_ADA_002, TEXT_EMBEDDING_ADA_3_SMALL],
                          "EASTUS": [GPT_35_TURBO_16K, GPT_35_TURBO_INSTRUCT, GPT_4o, GPT_4o_mini, DALL_E_3, TEXT_EMBEDDING_ADA_002, TEXT_EMBEDDING_ADA_3_LARGE],
                          "NORTHCENTRALUS": [WHISPER, GPT_4o]}


# carregar tokenizador para os modelos de linguagem
encoding = tiktoken.get_encoding("cl100k_base")

# inicialização do cliente Azure OpenAI Canada East
client_canadaeast = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_CANADAEAST"), 
    api_key=os.getenv("AZURE_OPENAI_KEY_CANADAEAST"),  
    api_version="2024-02-15-preview"
)

# inicialização do cliente Azure OpenAI East US
client_eastus = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_EASTUS"), 
    api_key=os.getenv("AZURE_OPENAI_KEY_EASTUS"),  
    api_version="2024-02-15-preview"
)

# inicialização do cliente Azure OpenAI North Central US
client_northcentralus = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_NORTHCENTRALUS"), 
    api_key=os.getenv("AZURE_OPENAI_KEY_NORTHCENTRALUS"),  
    api_version=os.getenv("WHISPER_VERSION")
)


# funcao para retornar o cliente de acordo com a região
def get_client(region):
    if region == "canadaeast":
        return client_canadaeast
    elif region == "eastus":
        return client_eastus
    elif region == "northcentralus":
        return client_northcentralus
    else:
        return None

# função para chamar o modelo de linguagem
def call_llm(deployment_name, system_message, prompt, file_path, region="canadaeast"):

    if deployment_name not in deployments_in_regions[region.upper()]:
        return {"0.error": f"Model {deployment_name} not available in region {region}", 
                "1.This regions has the following models available": deployments_in_regions[region.upper()], 
                f"2.The model {deployment_name} is available in the following regions": [region for region, models in deployments_in_regions.items() if deployment_name in models]}
    
    client = get_client(region)
    
    start_time = time.perf_counter()
    
    messages = [{"role": "system", "content": system_message},{"role": "user", "content": prompt}]
    
    if file_path:
        print("file_path", file_path)
        messages = [{"role": "system", "content": system_message},
                    {"role": "user", "content": [{"type": "text", "text": prompt}, get_content_file(file_path)]}]
    
    print("messages", messages)
    response = client.chat.completions.create(
        model = deployment_name,
        messages = messages
    )
    elapsed_time = time.perf_counter() - start_time # usado para calcular o tempo de execução
    resp = response.choices[0].message.content
    tokens_completion = num_tokens_from_string(resp, deployment_name) # conta os tokens da resposta
    # conta os tokens da mensagem do sistema e da pergunta (prompt)
    tokens_prompt = num_tokens_from_string(system_message, deployment_name) + num_tokens_from_string(prompt, deployment_name)
    # formatar a resposta da função
    return {"0.model": deployment_name, 
            "1.elapsed_time": elapsed_time, 
            "2.response": resp, 
            "3.num_tokens_completion": tokens_completion, 
            "4.num_tokens_prompt": tokens_prompt,
            "5.system_message": system_message,
            "6.prompt": prompt}

def get_content_file(file_path):
    response = {}
    if file_path.endswith(".wav"):
        with open(file_path, "rb") as audio_file:
            response = {
              "type": "audio_url",
              "audio_url": {
                "url": f"data:audio/wav;base64,{base64.b64encode(audio_file.read()).decode()}",
                "detail": "auto"
              }
            }
    elif file_path.endswith(".jpg") or file_path.endswith(".jpeg") or file_path.endswith(".png"):
        with open(file_path, "rb") as image_file:
            response = {
                    "type": "image_url",
                    "image_url": {
                        "url":  f"data:image/jpeg;base64,{base64.b64encode(image_file.read()).decode('utf-8')}",
                        "detail": "auto"
                    }
                }
    elif file_path.endswith(".pdf"):
        with open(file_path, "rb") as pdf_file:
            response = {
                "type": "text",
                "file": {
                    "url": f"data:text/pdf;base64,{base64.b64encode(pdf_file.read()).decode()}",
                    "detail": "auto"
                }
            }
    else:
        response = {
            "type": "file",
            "file": {
                "url": f"data:file/octet-stream;base64,{base64.b64encode(open(file_path, 'rb').read()).decode()}",
                "detail": "auto"
            }
        }
    print(response)
    return response

# funcao para contar tokens
def num_tokens_from_string(texto, model):
    if not texto:
        return 0
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(texto))
    return num_tokens

# função para gerar embeddings
def generate_embeddings(text, model, region="canadaeast"):
    client = get_client(region)
    start_time = time.perf_counter()
    embeddings = client.embeddings.create(input = [text], model=model).data[0].embedding
    elapsed_time = time.perf_counter() - start_time
    return embeddings, elapsed_time

# função para executar modelos simultaneamente
# models: lista de modelos
# system_message: mensagem do sistema
# prompt: pergunta
# retorna uma lista com os resultados
def execute_simultaneously(models, system_message, prompt, debug=False, region="canadaeast"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(call_llm, model, system_message, prompt, None, region) for model in models]
        results = []
        for future in concurrent.futures.as_completed(futures):
            if debug:
                print_result(future.result(), debug)
            results.append(future.result())
    return results

def print_result(result, debug=False):
    print("Elapsed time:", result["1.elapsed_time"])
    print("Response:", result["2.response"])
    print("Num tokens completion:", result["3.num_tokens_completion"])
    print("Num tokens prompt:", result["4.num_tokens_prompt"])
    if debug:
        print("System message:", result["5.system_message"])
        print("Prompt:", result["6.prompt"])
        print("Model:", result["0.model"])
        print("===========================================================\n")  

#funcao para imprimir resultados
def print_results(system_message, prompt, results, debug=False):
    if debug:
        print("System message:", system_message)
        print("Prompt:", prompt)

    for result in results:
        print("Model:", result["0.model"], "Elapsed time:", result["1.elapsed_time"])
        print("Response:", result["2.response"])
        print("Num tokens completion:", result["3.num_tokens_completion"], 
              ", Num tokens prompt:", result["4.num_tokens_prompt"])
        print("===========================================================\n")

# funcao para criar arquivo temporario
def create_temp_file(content, file_extension):
    with tempfile.NamedTemporaryFile(delete=False, suffix=file_extension) as temp_file:
        temp_file.write(content.encode())
        temp_file_path = temp_file.name
    return temp_file_path

# funcao para executar arquivo python externo
def execute_external_script(script_path, *args):
    command = ["python", script_path] + list(args)
    result = subprocess.run(command, capture_output=True, text=True)
    return result.stdout, result.stderr, result.returncode

# funcao para executar arquivo python temporario
def execute_temp_script(script_content, *args):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".py") as temp_script:
        temp_script.write(script_content.encode())
        temp_script_path = temp_script.name

    command = ["python", temp_script_path] + list(args)
    result = subprocess.run(command, capture_output=True, text=True)

    # Remover o arquivo temporário após a execução
    os.remove(temp_script_path)

    return result.stdout, result.stderr, result.returncode

# executa uma função python que está em uma string
def execute_function_from_string(func_code, *args):
    func_name = re.search(r"def (\w+)\(", func_code).group(1)
    exec(func_code)
    func = locals()[func_name]
    return func(*args)

# valida se a string contem um json válido se conter retorna o json
def hold_llm_json_response(json_string):
    # remove o prefixo "```json" se existir
    json_string = re.sub(r"^```json\s*", "", json_string, flags=re.MULTILINE)
    # remove o sufixo "```" se existir
    json_string = re.sub(r"```$", "", json_string, flags=re.MULTILINE)
    if isinstance(json_string, str):
        try:
            json_string = json.loads(json_string)
            return json_string
        except json.JSONDecodeError:
            print("Erro ao decodificar o JSON:", json_string)
            json_string = []
    else:
        print("Erro: a resposta não é uma string JSON válida.")


In [22]:
df_campaign = pd.read_csv("../data/ad_campaign_performance.csv")
colunas = df_campaign.columns
print(colunas)

Index(['Campaign_ID', 'Budget', 'Duration', 'Platform', 'Content_Type',
       'Target_Age', 'Target_Gender', 'Region', 'Clicks', 'Conversions', 'CTR',
       'CPC', 'Conversion_Rate', 'Success'],
      dtype='object')


In [45]:
prompt = f'''
Possuo um dataset com as seguintes colunas: 
{colunas}

Quais insights posso obter a partir deste dataset?
Gere um json com os seguintes campos:
- insights: lista de insights: cada insight deve ser uma string com o insight, uma função python que gera o insight e uma lista de colunas que foram utilizadas para gerar o insight.

Todas as funções python devem ser executáveis e devem retornar o insight. Elas devem receber como parâmetro o dataframe e retornar o insight.
retorne apenas o json, sem explicações adicionais.
Não inclua nenhum outro texto ou explicação.
'''

system_message = '''
    Você é um especialista em marketing digital e análise de dados.
    Seu trabalho é analisar dados de campanhas de marketing e fornecer insights valiosos.

'''

r = call_llm(GPT_4o, system_message, prompt, None, EASTUS)

insights = r["2.response"]
insights = hold_llm_json_response(insights)

for insight in insights["insights"]:
    insight_code = insight["function"]
    print("===========================================================\n")
    print(insight["insight"])
    r = execute_function_from_string(insight_code, df_campaign)
    print("Resultado:", r)
    print("===========================================================\n")


messages [{'role': 'system', 'content': '\n    Você é um especialista em marketing digital e análise de dados.\n    Seu trabalho é analisar dados de campanhas de marketing e fornecer insights valiosos.\n\n'}, {'role': 'user', 'content': "\nPossuo um dataset com as seguintes colunas: \nIndex(['Campaign_ID', 'Budget', 'Duration', 'Platform', 'Content_Type',\n       'Target_Age', 'Target_Gender', 'Region', 'Clicks', 'Conversions', 'CTR',\n       'CPC', 'Conversion_Rate', 'Success'],\n      dtype='object')\n\nQuais insights posso obter a partir deste dataset?\nGere um json com os seguintes campos:\n- insights: lista de insights: cada insight deve ser uma string com o insight, uma função python que gera o insight e uma lista de colunas que foram utilizadas para gerar o insight.\n\nTodas as funções python devem ser executáveis e devem retornar o insight. Elas devem receber como parâmetro o dataframe e retornar o insight.\nretorne apenas o json, sem explicações adicionais.\nNão inclua nenhum 

In [13]:
prompt = f'''
Escreva uma função em Python para calcular o primeiro insight desta lista de insights: {insights}
para um dataset com as seguintes colunas: {colunas}
'''

system_message = '''
    Você é um especialista em Python e pode escrever funções para calcular insights a partir de datasets.
'''
r = call_llm(GPT_4o, system_message, prompt, None, EASTUS)
print(r["2.response"])

messages [{'role': 'system', 'content': '\n    Você é um especialista em Python e pode escrever funções para calcular insights a partir de datasets.\n'}, {'role': 'user', 'content': "\nEscreva uma função em Python para calcular o primeiro insight desta lista de insights: Com esse dataset, você pode analisar uma variedade de aspectos das campanhas de marketing para identificar tendências, comparar eficácia e otimizar futuras estratégias. Aqui estão alguns insights potenciais que você pode obter:\n\n1. **Desempenho por Campanha**:\n   - Compare campanhas com base no sucesso, taxa de conversão (Conversion_Rate) e custo por clique (CPC) para identificar quais foram mais eficazes.\n\n2. **Efetividade do Orçamento**:\n   - Analise a relação entre Budget (orçamento) e Conversions (conversões). Veja se há um ponto de saturação onde aumentar o orçamento não melhora significativamente as conversões.\n\n3. **Desempenho por Plataforma**:\n   - Compare o desempenho (CTR, CPC, taxa de conversão) ent

In [15]:
def analyze_campaign_performance(df):
    """
    Analisa o desempenho das campanhas com base em sucesso, taxa de conversão e custo por clique.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame Pandas contendo os dados da campanha.
        
    Retorna:
        pd.DataFrame: Um resumo das campanhas com suas métricas de sucesso.
    """
    # Agrupar dados por 'Campaign_ID' e calcular métricas de interesse
    campaign_performance = df.groupby('Campaign_ID').agg({
        'Success': 'mean',        # Média de sucesso (presumindo que Success é uma métrica numérica)
        'Conversion_Rate': 'mean', # Média da taxa de conversão
        'CPC': 'mean'              # Média do custo por clique
    }).reset_index()
    
    # Ordenar o dataframe para uma avaliação mais fácil
    campaign_performance_sorted = campaign_performance.sort_values(
        by=['Success', 'Conversion_Rate', 'CPC'], 
        ascending=[False, False, True]  # Para maximizar sucesso e taxa de conversão, mas minimizar CPC
    )
    
    return campaign_performance_sorted

campaign_performance_sorted = analyze_campaign_performance(df_campaign)
campaign_performance_sorted

,Campaign_ID,Success,Conversion_Rate,CPC
413,CAMP-EFQJG0,1.0,734.513274,6.568584
399,CAMP-DW2PCN,1.0,592.173913,3.146584
393,CAMP-DR08CW,1.0,506.314433,15.612113
919,CAMP-XAA2IL,1.0,401.186624,11.143474
815,CAMP-TGSIWG,1.0,243.232115,14.130211
...,...,...,...,...
619,CAMP-MT8VT4,0.0,0.153906,1.010800
935,CAMP-XPNMJQ,0.0,0.139972,0.802373
108,CAMP-3OCWBF,0.0,0.125975,0.841858
778,CAMP-S13C9X,0.0,0.065245,0.042792


In [ ]:
def sample_agenda():
    agenda = '''12/03/2022 12:00:00 Ocupado 12/03/2022 13:00:00 Ocupado 12/03/2022 14:00:00 Ocupado 12/03/2022 15:00:00 Livre 12/03/2022 16:00:00 Ocupado 12/03/2022 17:00:00 Livre
        12/03/2022 18:00:00 Ocupado 12/03/2022 19:00:00 Livre 12/03/2022 20:00:00 Ocupado'''
    question = f"Com base nessa agenda, qual o próximo horário livre? {agenda}"
    return call_llm(deployment_name=GPT_35_TURBO, 
                    system_message="Aja como um assistente pessoal e ajude-me a organizar minha agenda.", 
                    prompt=question, 
                    file_path=None, 
                    region=CANADAEAST)

resposta = sample_agenda()
print_result(resposta)


messages [{'role': 'system', 'content': 'Aja como um assistente pessoal e ajude-me a organizar minha agenda.'}, {'role': 'user', 'content': 'Com base nessa agenda, qual o próximo horário livre? 12/03/2022 12:00:00 Ocupado 12/03/2022 13:00:00 Ocupado 12/03/2022 14:00:00 Ocupado 12/03/2022 15:00:00 Livre 12/03/2022 16:00:00 Ocupado 12/03/2022 17:00:00 Livre\n        12/03/2022 18:00:00 Ocupado 12/03/2022 19:00:00 Livre 12/03/2022 20:00:00 Ocupado'}]
Elapsed time: 1.8564947999998367
Response: O próximo horário livre disponível é às 15:00 do dia 12/03/2022. Gostaria de agendar alguma atividade ou compromisso para esse horário?
Num tokens completion: 42
Num tokens prompt: 174


# Exemplo de diferentes tons na resposta

In [ ]:
def sample_system_message_intonation(question):
    system_message = {"despojado":"Você é um assitente que sempre responde em tom despojado, de forma didática e em porguguês, apesar de eventualmente citar palavras em inglês.",
                      "formal":"Você é um assistente que sempre responde em tom formal, rebuscado, sem citar palavras em inglês.",
                      "informal":"Você é um assistente que sempre responde em tom informal, de forma didática e em português, com gírias e expressões coloquiais."}
    

    for item in system_message:
        print(f"Resposta com tom {item.upper()}:")
        r = call_llm(GPT_35_TURBO, system_message[item], question, None, "canadaeast")
        pprint.pprint(r["2.response"])

sample_system_message_intonation("O que é inteligência artificial?")

# Exemplo de diferença entre modelos

In [ ]:
system_message = '''Aja como um professor de lógica e explique-me conceitos fundamentais da lógica de 
maneira estruturada e sucinta com o mínimo de detalhes possível.'''
question = "Conte-me um paradoxo."
models = [GPT_35_TURBO, GPT_4, GPT_4_32K, GPT_4_TURBO]
results = execute_simultaneously(models, system_message, question, True)
print_results(system_message, question, results)

# Extraindo informações de uma imagem

In [ ]:
# faça uma pergunta ao modelo, seja criativo!
system_message = "Aja com um assitente pessoal e ajude-me a analizar o arquivo anexo."
file_path = "../data/img1.jpg"

prompt = ''' 

do que se trata o arquivo?

'''

r = call_llm(GPT_4o, system_message, prompt, file_path, EASTUS)["2.response"]
print(r)

In [ ]:
# faça uma pergunta ao modelo, seja criativo!
system_message = "Aja com um assitente pessoal e ajude-me a analizar o arquivo anexo."

file_path = "../data/emails_tels.jpg"

prompt = ''' 

extraia os dados da imagem em formato JSON

'''

r = call_llm(GPT_4o_mini, system_message, prompt, file_path, EASTUS)["2.response"]
print(r)

In [ ]:
a = 1
b = 2
prompt = f'''

def sum(a, b):
    return a + b

sum({a}, {b})

'''
system_message = "Aja como um interprete Python e execute o código abaixo."
file_path = None
r = call_llm(GPT_4o, system_message, prompt, file_path, EASTUS)["2.response"]
print(r) 